# Prison population of England and Wales

## Extracting and combining population data from downloaded Gov.uk files

In [1]:
import glob
import pandas as pd
import re

In [2]:
df = pd.read_excel("data/raw/2024/population_19_July_2024.ods", engine="odf")
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,2024-07-19 00:00:00
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Population and Capacity Briefing for Friday 19...,NaN,NaN,NaN,`,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,This week,NaN,NaN
4,NaN,NaN,Population,NaN,NaN,87343,NaN,NaN
5,NaN,NaN,Population in male estate,NaN,NaN,83692,NaN,NaN
6,NaN,NaN,Population in female estate,NaN,NaN,3651,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,Useable Operational Capacity,NaN,NaN,88848,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = (
    pd.read_excel("data/raw/2024/population_19_July_2024.ods", 
                nrows=8, 
                skiprows=[1, 2, 3, 4],
                usecols="C,F,H",
                engine="odf")
    .dropna(how="all") # drop rows that are all NA
)
df

,Unnamed: 2,Unnamed: 5,2024-07-19 00:00:00
0,Population,87343.0,NaN
1,Population in male estate,83692.0,NaN
2,Population in female estate,3651.0,NaN
4,Useable Operational Capacity,88848.0,NaN
6,Home Detention Curfew caseload,2449.0,NaN


In [ ]:
df.iloc[:, -1]

In [4]:
df['date'] = df.columns[-1].date()
df

,Unnamed: 2,Unnamed: 5,2024-07-19 00:00:00,date
0,Population,87343.0,NaN,2024-07-19
1,Population in male estate,83692.0,NaN,2024-07-19
2,Population in female estate,3651.0,NaN,2024-07-19
4,Useable Operational Capacity,88848.0,NaN,2024-07-19
6,Home Detention Curfew caseload,2449.0,NaN,2024-07-19


In [5]:
df.drop(columns=df.columns[-2], inplace=True)

In [6]:
df

,Unnamed: 2,Unnamed: 5,date
0,Population,87343.0,2024-07-19
1,Population in male estate,83692.0,2024-07-19
2,Population in female estate,3651.0,2024-07-19
4,Useable Operational Capacity,88848.0,2024-07-19
6,Home Detention Curfew caseload,2449.0,2024-07-19


In [7]:
df.columns = ["type", "population", "date"]
df

,type,population,date
0,Population,87343.0,2024-07-19
1,Population in male estate,83692.0,2024-07-19
2,Population in female estate,3651.0,2024-07-19
4,Useable Operational Capacity,88848.0,2024-07-19
6,Home Detention Curfew caseload,2449.0,2024-07-19


In [9]:
df["population"] = df["population"].astype(int)
df

,type,population,date
0,Population,87343,2024-07-19
1,Population in male estate,83692,2024-07-19
2,Population in female estate,3651,2024-07-19
4,Useable Operational Capacity,88848,2024-07-19
6,Home Detention Curfew caseload,2449,2024-07-19


In [10]:
df.replace({
    "Population": "total",
    "Population in male estate": "male",
    "Population in female estate": "female",
    "Useable Operational Capacity": "operational capacity",
    "Home Detention Curfew caseload": "HDC"
    }, 
    inplace=True)

df

,type,population,date
0,total,87343,2024-07-19
1,male,83692,2024-07-19
2,female,3651,2024-07-19
4,operational capacity,88848,2024-07-19
6,HDC,2449,2024-07-19


### Consolidating code up to this point

In [12]:
df = (
    pd.read_excel(
        "data/raw/2024/population_19_July_2024.ods",
        nrows=8,
        skiprows=[1, 2, 3, 4],
        usecols="C,F,H",
        engine="odf"
    )
    .dropna(how="all")  # Drop rows that are all NA
    .assign(date=lambda x: x.columns[-1].date())  # Extract date from last column name
    .pipe(lambda x: x.drop(columns=[x.columns[-2]]))  # Drop second-to-last column correctly
    .set_axis(["type", "population", "date"], axis=1)  # Rename columns
    .astype({"population": int})  # Convert population to integer
    .replace({
        "Population": "total",
        "Population in male estate": "male",
        "Population in female estate": "female",
        "Useable Operational Capacity": "operational capacity",
        "Home Detention Curfew caseload": "HDC"
    })
)
df

,type,population,date
0,total,87343,2024-07-19
1,male,83692,2024-07-19
2,female,3651,2024-07-19
4,operational capacity,88848,2024-07-19
6,HDC,2449,2024-07-19


## Applying processing to a folder of data

In [ ]:
import pandas as pd
import glob

In [14]:
# Define the folder path and file pattern
folder_path = "data/raw/2024/"
file_pattern = "*.ods"

# Get all matching file paths
file_paths = glob.glob(f"{folder_path}/{file_pattern}")

# Define a function to process a single file
def process_file(file_path):
    try:
        df = pd.read_excel(
            file_path,
            nrows=8,
            skiprows=[1, 2, 3, 4],
            usecols="C,F,H",
            engine="odf"
        ).dropna(how="all")  # Drop rows that are all NA

        # Debugging: Print column names before extracting the date
        print(f"Processing {file_path}, Columns: {df.columns.tolist()}")

        # Ensure the last column is a datetime-like value
        last_col = df.columns[-1]
        if not isinstance(last_col, pd.Timestamp):  # Check if it's a timestamp
            raise ValueError(f"Unexpected column format in {file_path}: {last_col}")

        return (
            df.assign(date=last_col.date())  # Extract date from last column name
            .pipe(lambda x: x.drop(columns=[x.columns[-2]]))  # Drop second-to-last column
            .set_axis(["type", "population", "date"], axis=1)  # Rename columns
            .astype({"population": int})  # Convert population to integer
            .replace({
                "Population": "total",
                "Population in male estate": "male",
                "Population in female estate": "female",
                "Useable Operational Capacity": "operational capacity",
                "Home Detention Curfew caseload": "HDC"
            })
        )
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None  # Skip problematic files

# Process all files and concatenate into one DataFrame
df_all = pd.concat([process_file(file) for file in file_paths], ignore_index=True)

# Display the final DataFrame
df_all


Processing data/raw/2024/prison-pop-9-aug-2024.ods, Columns: ['Unnamed: 2', 'Unnamed: 5', Timestamp('2024-08-09 00:00:00')]
Processing data/raw/2024/Prison_population_report_23_Feb_24.ods, Columns: ['Unnamed: 2', 'Unnamed: 5', Timestamp('2024-02-23 00:00:00')]
Processing data/raw/2024/prison-pop-24-may-2024.ods, Columns: ['Unnamed: 2', 'Unnamed: 5', Timestamp('2024-05-24 00:00:00')]
Processing data/raw/2024/prison-pop-15-march-2024.ods, Columns: ['Unnamed: 2', 'Unnamed: 5', Timestamp('2024-03-15 00:00:00')]
Processing data/raw/2024/prison-population-31-May.ods, Columns: ['Unnamed: 2', 'Unnamed: 5', Timestamp('2024-05-31 00:00:00')]
Processing data/raw/2024/prison-pop-26-jan-2024.ods, Columns: ['Unnamed: 2', 'Unnamed: 5', Timestamp('2024-01-26 00:00:00')]
Processing data/raw/2024/prison-pop-16-dec-2024.ods, Columns: ['Unnamed: 2', 'Unnamed: 5', 'Unnamed: 7']
Error processing data/raw/2024/prison-pop-16-dec-2024.ods: Unexpected column format in data/raw/2024/prison-pop-16-dec-2024.ods: U

,type,population,date
0,total,87496,2024-08-09
1,male,83900,2024-08-09
2,female,3596,2024-08-09
3,operational capacity,88936,2024-08-09
4,HDC,2645,2024-08-09
...,...,...,...
200,total,87691,2024-05-10
201,male,84049,2024-05-10
202,female,3642,2024-05-10
203,operational capacity,88929,2024-05-10


In [26]:
sorted(df_all['date'].unique())

[datetime.date(2024, 1, 5),
 datetime.date(2024, 1, 12),
 datetime.date(2024, 1, 19),
 datetime.date(2024, 1, 26),
 datetime.date(2024, 2, 2),
 datetime.date(2024, 2, 9),
 datetime.date(2024, 2, 16),
 datetime.date(2024, 2, 23),
 datetime.date(2024, 3, 1),
 datetime.date(2024, 3, 8),
 datetime.date(2024, 3, 15),
 datetime.date(2024, 3, 22),
 datetime.date(2024, 4, 5),
 datetime.date(2024, 4, 12),
 datetime.date(2024, 4, 19),
 datetime.date(2024, 4, 26),
 datetime.date(2024, 5, 3),
 datetime.date(2024, 5, 10),
 datetime.date(2024, 5, 17),
 datetime.date(2024, 5, 24),
 datetime.date(2024, 5, 31),
 datetime.date(2024, 6, 7),
 datetime.date(2024, 6, 14),
 datetime.date(2024, 6, 21),
 datetime.date(2024, 6, 28),
 datetime.date(2024, 7, 5),
 datetime.date(2024, 7, 12),
 datetime.date(2024, 7, 19),
 datetime.date(2024, 7, 26),
 datetime.date(2024, 8, 2),
 datetime.date(2024, 8, 9),
 datetime.date(2024, 8, 16),
 datetime.date(2024, 8, 23),
 datetime.date(2024, 8, 30),
 datetime.date(2024, 9, 6

In [22]:
len(df_all['date'].unique())

41

Right, I have an issue with the format of the spreadsheets changing. So I now need to investigate for differences that can be used to differentiate the type of file being processed, and adjust the processing that it goes through.

## Differentiating historic and new file formatting

In [ ]:
df_new = (
    pd.read_excel("data/raw/2024/prison-pop-16-dec-2024.ods", 
                # nrows=8, 
                # skiprows=[1, 2, 3, 4],
                # usecols="C,F,H",
                engine="odf")
    .dropna(how="all") # drop rows that are all NA
)
df_new

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
1,NaN,NaN,Population and Capacity Briefing for Monday 16...,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,Total,NaN,Adult Male,Female,YCS
4,NaN,NaN,Population,NaN,85960,NaN,82114,3485,361
5,NaN,NaN,Useable Operational Capacity,NaN,88692,NaN,84387,3853,452
6,NaN,NaN,Headroom,NaN,2732,NaN,2273,368,91
8,NaN,NaN,Home Detention Curfew caseload,NaN,4101,NaN,NaN,NaN,NaN
10,NaN,NaN,Population and Capacity on previous Mondays,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,Last Week: 09 December 2024,NaN,Total,NaN,Adult Male,Female,YCS
13,NaN,NaN,Population,NaN,86089,NaN,82218,3500,371
14,NaN,NaN,Useable Operational Capacity,NaN,88822,NaN,84517,3853,452


In [35]:
df_old = (
    pd.read_excel("data/raw/2024/population_19_July_2024.ods", 
                # nrows=8, 
                # skiprows=[1, 2, 3, 4],
                # usecols="C,F,H",
                engine="odf")
    .dropna(how="all") # drop rows that are all NA
)
df_old

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,2024-07-19 00:00:00
1,NaN,NaN,Population and Capacity Briefing for Friday 19...,NaN,NaN,NaN,`,NaN
3,NaN,NaN,NaN,NaN,NaN,This week,NaN,NaN
4,NaN,NaN,Population,NaN,NaN,87343,NaN,NaN
5,NaN,NaN,Population in male estate,NaN,NaN,83692,NaN,NaN
6,NaN,NaN,Population in female estate,NaN,NaN,3651,NaN,NaN
8,NaN,NaN,Useable Operational Capacity,NaN,NaN,88848,NaN,NaN
10,NaN,NaN,Home Detention Curfew caseload,NaN,NaN,2449,NaN,NaN
12,NaN,NaN,Definitions:,NaN,NaN,NaN,NaN,NaN
14,NaN,NaN,1 - The Operational Capacity of a Prison is th...,NaN,NaN,NaN,NaN,NaN
16,NaN,NaN,2 - Useable Operational Capacity of the estate...,NaN,NaN,NaN,NaN,NaN


In [36]:
df_new.shape

(25, 9)

In [37]:
df_old.shape

(18, 8)

In [33]:
import pandas as pd
import glob
from collections import Counter

folder_path = "data/raw/2024/"  # Adjust this to your actual folder
file_paths = glob.glob(f"{folder_path}/*.ods")  # Adjust extension if needed

shape_counts = Counter()

for file in file_paths:
    try:
        df = pd.read_excel(file, engine="odf").dropna(how="all")  # Read full file
        shape_counts[df.shape] += 1
    except Exception as e:
        print(f"Error reading {file}: {e}")

# Print results
print("Unique file shapes and their counts:")
for shape, count in shape_counts.items():
    print(f"Shape {shape}: {count} files")


Unique file shapes and their counts:
Shape (18, 8): 41 files
Shape (25, 9): 11 files


Great, so I now know that — within this folder at least — there are only two unique dataframe shapes. I can now use this to difference to identify how the file formats should be processed. 

### Testing

Checking the new format to determine which bits I need.

In [ ]:
df_new = pd.read_excel("data/raw/2024/prison-pop-16-dec-2024.ods", engine="odf")
df_new

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Population and Capacity Briefing for Monday 16...,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,Total,NaN,Adult Male,Female,YCS
4,NaN,NaN,Population,NaN,85960,NaN,82114,3485,361
5,NaN,NaN,Useable Operational Capacity,NaN,88692,NaN,84387,3853,452
6,NaN,NaN,Headroom,NaN,2732,NaN,2273,368,91
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,Home Detention Curfew caseload,NaN,4101,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


There doesn't appear to be any timestamp values in here, unlike the old format. So I'm going to need to use some regex to extract the date from the correct row and column.

Let's isolate all the bits that I need from the spreadsheet first and then move on to extracting the date.

In [61]:
df_new = (
    pd.read_excel("data/raw/2024/prison-pop-16-dec-2024.ods", engine="odf")
    .dropna(how="all") # drop rows that are all NA
)
df_new.iloc[list(range(6)), :]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
1,NaN,NaN,Population and Capacity Briefing for Monday 16...,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,Total,NaN,Adult Male,Female,YCS
4,NaN,NaN,Population,NaN,85960,NaN,82114,3485,361
5,NaN,NaN,Useable Operational Capacity,NaN,88692,NaN,84387,3853,452
6,NaN,NaN,Headroom,NaN,2732,NaN,2273,368,91
8,NaN,NaN,Home Detention Curfew caseload,NaN,4101,NaN,NaN,NaN,NaN


In [62]:
df_new.iloc[0, 2]

'Population and Capacity Briefing for Monday 16 December 2024'

In [63]:
import re
from datetime import datetime

In [58]:
def extract_date_from_string(date_string):
    """Extracts a date from a string using regex and converts it to a datetime object."""
    match = re.search(r"(\d{1,2} [A-Za-z]+ \d{4})", date_string)  # Looks for '19 July 2024'
    if match:
        return datetime.strptime(match.group(1), "%d %B %Y").date()
    return None

In [59]:
date_value = extract_date_from_string(df_new.iloc[0, 2])  # Extract date from cell
date_value

datetime.date(2024, 12, 16)

Great, that's worked as expected. Let's see if this works across all files.

In [68]:
# Define the folder path and file pattern
folder_path = "data/raw/2024/"
file_pattern = "*.ods"

# Get all matching file paths
file_paths = glob.glob(f"{folder_path}/{file_pattern}")

def process_file(file_path):
    try:
        # Read the file
        df = pd.read_excel(file_path, engine="odf").dropna(how="all")

        # Determine format based on shape
        shape = df.shape
        if shape == (18, 8):
            format_type = "historic"
            selected_rows = list(range(7))  # First 7 rows contain relevant data
            selected_cols = [2, 5]  # Adjust based on actual needed columns
            date_value = extract_date_from_string(df.iloc[0, 2])  # Extract date from cell
        elif shape == (25, 9):
            format_type = "new"
            selected_rows = list(range(6))  # First 6 rows contain relevant data
            selected_cols = [2, 4, 6, 7, 8]  # Adjust for new format
            date_value = extract_date_from_string(df.iloc[0, 2])  # Extract date from cell
            if date_value is None:
                raise ValueError(f"Could not extract date from {file_path}")
        else:
            raise ValueError(f"Unexpected format in {file_path}: {shape}")
        
        print(f"Processing {date_value} ({format_type} format)")
    
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None  # Skip problematic files

In [69]:
for file in file_paths:
    process_file(file)

Processing 2024-08-09 (historic format)
Processing 2024-02-23 (historic format)
Processing 2024-05-24 (historic format)
Processing 2024-03-15 (historic format)
Processing 2024-05-31 (historic format)
Processing 2024-01-26 (historic format)
Processing 2024-12-16 (new format)
Processing 2024-12-23 (new format)
Processing 2024-06-07 (historic format)
Processing 2024-10-18 (historic format)
Processing 2024-08-30 (historic format)
Processing 2024-07-19 (historic format)
Processing 2024-08-16 (historic format)
Processing 2024-03-01 (historic format)
Processing 2024-10-28 (new format)
Processing 2024-10-04 (historic format)
Processing 2024-11-11 (new format)
Processing 2024-03-08 (historic format)
Processing 2024-10-21 (new format)
Processing 2024-06-21 (historic format)
Processing 2024-02-09 (historic format)
Processing 2024-04-19 (historic format)
Processing 2024-08-23 (historic format)
Processing 2024-08-02 (historic format)
Processing 2024-04-05 (historic format)
Processing 2024-02-16 (hi

Extraction of dates is working perfectly by the looks of things. Now to double check which bits of the historic format files I need.

In [75]:
df_old = (
    pd.read_excel("data/raw/2024/population_19_July_2024.ods", engine="odf")
    .dropna(how="all") # drop rows that are all NA
)
df_old.iloc[list(range(7)), :]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,2024-07-19 00:00:00
1,NaN,NaN,Population and Capacity Briefing for Friday 19...,NaN,NaN,NaN,`,NaN
3,NaN,NaN,NaN,NaN,NaN,This week,NaN,NaN
4,NaN,NaN,Population,NaN,NaN,87343,NaN,NaN
5,NaN,NaN,Population in male estate,NaN,NaN,83692,NaN,NaN
6,NaN,NaN,Population in female estate,NaN,NaN,3651,NaN,NaN
8,NaN,NaN,Useable Operational Capacity,NaN,NaN,88848,NaN,NaN
10,NaN,NaN,Home Detention Curfew caseload,NaN,NaN,2449,NaN,NaN


In [88]:
# Define the folder path and file pattern
folder_path = "data/raw/2024/"
file_pattern = "*.ods"

# Get all matching file paths
file_paths = glob.glob(f"{folder_path}/{file_pattern}")

def extract_date_from_string(date_string):
    """Extracts a date from a string using regex and converts it to a datetime object."""
    match = re.search(r"(\d{1,2} [A-Za-z]+ \d{4})", date_string)  # Looks for '19 July 2024'
    if match:
        return datetime.strptime(match.group(1), "%d %B %Y").date()
    return None

def process_file(file_path):
    try:
        # Read the file
        df = pd.read_excel(file_path, engine="odf").dropna(how="all")

        # Determine format based on shape
        shape = df.shape
        if shape == (18, 8):
            format_type = "historic"
            selected_rows = list(range(7))  # First 7 rows contain relevant data
            selected_cols = [2, 5]  # Adjust based on actual needed columns
        elif shape == (25, 9):
            format_type = "new"
            selected_rows = list(range(6))  # First 6 rows contain relevant data
            selected_cols = [2, 4, 6, 7, 8]  # Adjust for new format
            if date_value is None:
                raise ValueError(f"Could not extract date from {file_path}")
        else:
            raise ValueError(f"Unexpected format in {file_path}: {shape}")

        print(f"Processing {file_path} as {format_type} format")

        # Extract relevant columns
        df = df.iloc[selected_rows, selected_cols]  # Select rows and columns

        return (
            df
            .assign(date=extract_date_from_string(df.iloc[0, 0]))  # Assign extracted date
            # .pipe(lambda x: x.drop(columns=[x.columns[-2]]))  # Drop last column before renaming
            # .set_axis(["type", "population", "date"], axis=1)
            # .astype({"population": int})
            # .replace({
            #     "Population": "total",
            #     "Population in male estate": "male",
            #     "Population in female estate": "female",
            #     "Useable Operational Capacity": "operational capacity",
            #     "Home Detention Curfew caseload": "HDC"
            # })
        )

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None  # Skip problematic files

In [90]:
test_new_df = process_file("data/raw/2024/prison-pop-16-dec-2024.ods")
test_new_df

Processing data/raw/2024/prison-pop-16-dec-2024.ods as new format


,Unnamed: 2,Unnamed: 4,Unnamed: 6,Unnamed: 7,Unnamed: 8,date
1,Population and Capacity Briefing for Monday 16...,NaN,NaN,NaN,NaN,2024-12-16
3,NaN,Total,Adult Male,Female,YCS,2024-12-16
4,Population,85960,82114,3485,361,2024-12-16
5,Useable Operational Capacity,88692,84387,3853,452,2024-12-16
6,Headroom,2732,2273,368,91,2024-12-16
8,Home Detention Curfew caseload,4101,NaN,NaN,NaN,2024-12-16


In [91]:
test_old_df = process_file("data/raw/2024/prison-pop-9-feb-2024.ods")
test_old_df

Processing data/raw/2024/prison-pop-9-feb-2024.ods as historic format


,Unnamed: 2,Unnamed: 5,date
1,Population and Capacity Briefing for Friday 09...,NaN,2024-02-09
3,NaN,This week,2024-02-09
4,Population,87982,2024-02-09
5,Population in male estate,84359,2024-02-09
6,Population in female estate,3623,2024-02-09
8,Useable Operational Capacity,89099,2024-02-09
10,Home Detention Curfew caseload,2189,2024-02-09


This seems to be working. Now need to pivot the new spreadsheet from wide to long, then sort out the column names.

In [55]:
# Process all files and concatenate into one DataFrame
df_all = pd.concat([process_file(file) for file in file_paths], ignore_index=True)

# Display the final DataFrame
df_all

Processing data/raw/2024/prison-pop-9-aug-2024.ods as historic format
Processing data/raw/2024/Prison_population_report_23_Feb_24.ods as historic format
Processing data/raw/2024/prison-pop-24-may-2024.ods as historic format
Processing data/raw/2024/prison-pop-15-march-2024.ods as historic format
Processing data/raw/2024/prison-population-31-May.ods as historic format
Processing data/raw/2024/prison-pop-26-jan-2024.ods as historic format
Processing data/raw/2024/prison-pop-16-dec-2024.ods as new format
Processing data/raw/2024/23_December_2024.ods as new format
Processing data/raw/2024/prison-pop-7-june-2024.ods as historic format
Processing data/raw/2024/prison-pop-18-october-2024.ods as historic format
Processing data/raw/2024/Prison_population_30_August.ods as historic format
Processing data/raw/2024/population_19_July_2024.ods as historic format
Processing data/raw/2024/prison-pop-16-aug-2024.ods as historic format
Processing data/raw/2024/prison-pop-1-march-2024.ods as historic for

,Unnamed: 2,Unnamed: 5,2024-08-09 00:00:00,date,2024-02-23 00:00:00,2024-05-24 00:00:00,2024-03-15 00:00:00,2024-05-31 00:00:00,2024-01-26 00:00:00,Unnamed: 4,...,2024-03-22 00:00:00,2024-05-03 00:00:00,2024-07-05 00:00:00,2024-04-12 00:00:00,2024-09-13 00:00:00,2024-04-26 00:00:00,2024-01-05 00:00:00,2024-01-12 00:00:00,2024-05-17 00:00:00,2024-05-10 00:00:00
0,Population and Capacity Briefing for Friday 09...,NaN,NaN,2024-08-09,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,This week,NaN,2024-08-09,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Population,87496,NaN,2024-08-09,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Population in male estate,83900,NaN,2024-08-09,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Population in female estate,3596,NaN,2024-08-09,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,Population,87691,NaN,2024-05-10,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
349,Population in male estate,84049,NaN,2024-05-10,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
350,Population in female estate,3642,NaN,2024-05-10,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
351,Useable Operational Capacity,88929,NaN,2024-05-10,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
